# How to get software on the SCRTP system

Documentation page - [Accessing and installing software](https://docs.scrtp.warwick.ac.uk/general-pages/software.html)

## Understanding Environments

Computing environments is the name we give to a collection of known software that we will use together to run some code or software program.
An environment will not use all the software on the machine, in fact there is a good chance you will have multiple versions of the same software on one machine.

To build an environment you need to isolate which programs and software you need.
To do this we will manipulate the available paths.
We discussed your PATH; it's a list of directories for the computer to search for programs.
There are other paths, for example: PYTHONPATH being a good example for python environment creation.

>> ### Thought Experiment
>>
>> If you are not using an environment then you likely don't know what programs could be being used.
>> However, thinking about versions really highlights how things can go wrong.
>>
>> Consider two packages A and B where B has A as a dependency.
>>
>> #### Without environments
>>
>> You have two projects 1 and 2.
>> 
>> You use A(V1) to generate a result for project 1.
>> 
>> For project 2 you install B to your computer.
>> B however requires a newer version of A(V2) so updates it.
>> 
>> During a review you need to recreate a result from project 1, however, as A has been updated to V2 by B the result is different because of some change made between A(V1) and A(V2).
>> 
>> #### With environments
>>
>> You have two projects 1 and 2.
>> 
>> You create a blank environment for project 1 and install A(V1) in it.
>> A result is created in this environment for project 1.
>>
>> You start a new project 2, creating a new blank environment for it.
>> In this environment you install B. It looks for A but as the environment is blank it can't find it and installs A(V2) which is its dependency.
>> You create a result for project 2.
>> 
>> During a review you need to recreate a result from project 1. 
>> Using the environment you created for project 1 you can recreate the result using A(V1) which has not been changed.
>> 
>> #### Conclusions
>>
>> This is example is slightly reductive and the results are usually more subtle.
>> But it should highlight how things can go wrong, most python environments have tens os packages which can all have complicated inter-dependencies on each other.
>> Package/environment managers have been built to manage this so we should use them.

## Modules

On the SCRTP systems we use the modules package.
Modules allows you to create an environment that contains all the software you need without the software you don't.
We have installed (or will install) the software you need and in many cases we install many many versions of common software. 
Lets see what we have:

```bash
module avail
```

or to search for something

```bash
module spider python
```

Modules is going to edit your environment path(s) by editing your paths we can point it to the correct version using your path.

Let's try and load Python for you to use, as opposed to system Python in `/usr/bin/`.
> Don't use the python in /usr/bin/python. This is the system python that the system uses and it does not have the ability to be extended with packages or used with other modules in the way you will want to use it!
> The system python may also be updated or modified during system maintenance
> You can tell which python is loaded using `which python`.

```bash 
module load Python/3.11.5
```
However this throws an error as Python has dependencies.
Furthermore different versions of Python will have different dependencies.
We can use module spider again to query this.

```bash
module spider Python/3.11.5
```

```bash
module spider Python/3.9.6
```

For 3.11.5 we need GCCcore/13.2.0 loaded and for 3.9.6 we need GCCcore/11.2.0

So to load Python/3.11.5 we run

```bash
module load GCCcore/13.2.0 Python/3.11.5
```

And now we have Python.

```bash
which python
```

And we can see all the modules we have loaded.

```bash
module list
```

Now while you can put modules to load in your .bash_profile this isn't recommended.
You should instead simplify loading and unloading using the module collections feature.
To save this python setup for use later we can create a collection using `module save`.

```bash
module save py3_11_env
```

Then next time you log onto the system you can quickly run.

```bash
module restore py3_11_env
```


## Now let's talk about building on-top of modules

The modules system is the way you should bring in programs that need to have been specifically compiled to run on the hardware. 
For example BLAS, however some language packages will use these as system dependencies.

### An example in Python

> Upfront, if you can avoid it don't use Conda/Miniconda as it plays much less nicely with building clean environments, if you _have_ to use it then do not use any modules provided Python.

The steps you should follow:

1. Load all your module dependencies using `module` load ensure compatibility.
2. Create a collection using `module save module_<env_name>`, you can name it however you like.
3. Create a Python environment using `python -m venv <env_name>`, again name it how you like.
4. Activate the env, `source <env_name>/bin/activate`
5. Install additional dependencies using `pip`, `pip install ____`
6. Check all expected libraries are available using `pip list`


> How to Debug: Deactivate env and unload modules, `deactivate` and `module purge`, then restore modules and source env `module restore <name>` and  reactivate env `source <name>/bin/activate`.
>
> Note: Do not load modules in the pip environment, if a new system module is desired the python env must be recreated from scratch.
>

Extra considerations, some python packages such as PyTorch have been specifically compiled to be optimized for our hardware.
Additionally, some bundles are provided for a quick `one and done` software stack in Python.
If you want to use these then you load them before creating your python environment.
Finally if a Python package, when installed by pip, has a system dependency that can be provided by modules this is advisable.

## Exercise: load a common python environment

This is a practice session for using these tools. 
If you are attending this course in person then you should also attempt to build an environment you will use in your research so we can help you.
If you are not in person consider using the SCRTP slack or Drop In to come and get advice on building your environment.

We want an environment with the following spec.

1. Python (modules latest possible)
2. Pytorch (from modules PIP-PyTorch)
3. Jupyter Notebook (from modules)
4. ASE (from modules)
5. cutensor-cu12 (from pip)


To validate this we will queue the following on a GPU reservation. (If you have Avon access)

```bash
[pfstcb@login01(avon) ~]$ module load GCC/11.3.0  OpenMPI/4.1.4 PIP-PyTorch/2.1.2

Due to MODULEPATH changes, the following have been reloaded:
  1) bzip2/1.0.8

The following have been reloaded with a version change:
  1) GCCcore/13.2.0 => GCCcore/11.3.0     5) XZ/5.4.4 => XZ/5.2.5                     9) ncurses/6.4 => ncurses/6.3
  2) Python/3.11.5 => Python/3.10.4       6) binutils/2.40 => binutils/2.38          10) zlib/1.2.13 => zlib/1.2.12
  3) SQLite/3.43.1 => SQLite/3.38.3       7) libffi/3.4.4 => libffi/3.4.2
  4) Tcl/8.6.13 => Tcl/8.6.12     
  ```

```bash
[pfstcb@login01(avon) ~]$ which python
/scrtp/avon/eb/software/Python/3.10.4-GCCcore-11.3.0/bin/python
```

```bash
[pfstcb@login01(avon) ~]$ module load CUDA/12.0.0
[pfstcb@login01(avon) ~]$ module list

Currently Loaded Modules:
  1) lmod               15) OpenBLAS/0.3.20     29) Python/3.10.4          43) SciPy-bundle/2022.05  57) gzip/1.12
  2) OpenSSL/1.1        16) FlexiBLAS/3.2.0     30) PIP-PyTorch/2.1.2      44) Flask/2.2.2           58) lz4/1.9.3
  3) GCCcore/11.3.0     17) FFTW/3.3.10         31) bzip2/1.0.8            45) libpng/1.6.37         59) zstd/1.5.2
  4) GCC/11.3.0         18) FFTW.MPI/3.3.10     32) OpenPGM/5.2.122        46) Brotli/1.0.9          60) libdeflate/1.10
  5) numactl/2.0.14     19) ScaLAPACK/2.2.0-fb  33) libsodium/1.0.18       47) freetype/2.12.1       61) LibTIFF/4.3.0
  6) libxml2/2.9.13     20) binutils/2.38       34) util-linux/2.38        48) expat/2.4.8           62) Pillow/9.1.1
  7) libpciaccess/0.16  21) ncurses/6.3         35) ZeroMQ/4.3.4           49) fontconfig/2.14.0     63) Qhull/2020.2
  8) hwloc/2.7.1        22) libreadline/8.1.2   36) libxslt/1.1.34         50) xorg-macros/1.19.3    64) matplotlib/3.5.2
  9) libevent/2.1.12    23) zlib/1.2.12         37) lxml/4.9.1             51) X11/20220504          65) spglib-python/2.0.0
 10) UCX/1.12.1         24) Tcl/8.6.12          38) BeautifulSoup/4.10.0   52) Tk/8.6.12             66) ASE/3.22.1
 11) libfabric/1.15.1   25) SQLite/3.38.3       39) IPython/8.5.0          53) Tkinter/3.10.4        67) CUDAcore/12.0.0
 12) PMIx/4.1.2         26) XZ/5.2.5            40) jupyter-server/1.21.0  54) NASM/2.15.05          68) CUDA/12.0.0
 13) UCC/1.0.0          27) GMP/6.2.1           41) JupyterLab/3.5.0       55) libjpeg-turbo/2.1.3
 14) OpenMPI/4.1.4      28) libffi/3.4.2        42) pybind11/2.9.2         56) jbigkit/2.1


 ```

```bash
[pfstcb@login01(avon) ~]$ module save mod_pyt_ase_jpy
Saved current collection of modules to: "mod_pyt_ase_jpy"

[pfstcb@login01(avon) ~]$ which python
/scrtp/avon/eb/software/Python/3.10.4-GCCcore-11.3.0/bin/python
[pfstcb@login01(avon) ~]$ python -m venv env_pyt_ase_jpy --system-site-packages
[pfstcb@login01(avon) ~]$ source env_pyt_ase_jpy/bin/activate
```

 ```bash
 (env_pyt_ase_jpy) [pfstcb@login01(avon) ~]$ which python
/gpfs/home/csc/pfstcb/env_pyt_ase_jpy/bin/python
```

```bash
(env_pyt_ase_jpy) [pfstcb@login01(avon) ~]$ pip list
Package                  Version
------------------------ -----------
anyio                    3.6.1
argon2-cffi              20.1.0
ase                      3.22.1
ase-ext                  20.9.0
asgiref                  3.5.2
asttokens                2.0.8
async-generator          1.10
backcall                 0.2.0
beautifulsoup4           4.10.0
beniget                  0.4.1
bleach                   5.0.1
Bottleneck               1.3.4
cachelib                 0.9.0
cycler                   0.11.0
deap                     1.3.1
debugpy                  1.4.1
defusedxml               0.7.1
deprecation              2.1.0
entrypoints              0.4
executing                1.0.0
fastjsonschema           2.16.1
Flask                    2.2.2
Flask-Cors               3.0.10
Flask-Session            0.4.0
fonttools                4.34.0
gast                     0.5.3
ipykernel                6.13.0
ipython                  8.5.0
ipython-genutils         0.2.0
ipywidgets               7.7.4
itsdangerous             2.1.2
jedi                     0.18.1
json5                    0.9.10
jupyter-client           7.3.1
jupyter-core             4.10.0
jupyter-packaging        0.12.0
jupyter-server           1.21.0
jupyterlab               3.5.0
jupyterlab-pygments      0.1.2
jupyterlab-server        2.13.0
jupyterlab-widgets       1.1.3
kiwisolver               1.4.3
lxml                     4.9.1
matplotlib               3.5.2
matplotlib-inline        0.1.2
mistune                  0.8.1
mpi4py                   3.1.3
mpmath                   1.2.1
nbclassic                0.4.8
nbclient                 0.6.3
nbconvert                6.5.3
nbformat                 5.4.0
nest-asyncio             1.5.5
networkx                 3.2.1
notebook                 6.4.0
notebook-shim            0.1.0
numexpr                  2.8.1
numpy                    1.22.3
nvidia-cublas-cu12       12.1.3.1
nvidia-cuda-cupti-cu12   12.1.105
nvidia-cuda-nvrtc-cu12   12.1.105
nvidia-cuda-runtime-cu12 12.1.105
nvidia-cudnn-cu12        8.9.2.26
nvidia-cufft-cu12        11.0.2.54
nvidia-curand-cu12       10.3.2.106
nvidia-cusolver-cu12     11.4.5.107
nvidia-cusparse-cu12     12.1.0.106
nvidia-nccl-cu12         2.18.1
nvidia-nvjitlink-cu12    12.3.101
nvidia-nvtx-cu12         12.1.105
pandas                   1.4.2
pandocfilters            1.5.0
parso                    0.8.3
pickleshare              0.7.5
Pillow                   9.1.1
pip                      22.0.4
ply                      3.11
prometheus-client        0.11.0
prompt-toolkit           3.0.31
pure-eval                0.2.2
pybind11                 2.9.2
pytest-mock              3.8.2
pythran                  0.11.0
pyzmq                    23.2.1
SciPy                    1.8.1
Send2Trash               1.8.0
setuptools               58.1.0
sniffio                  1.3.0
soupsieve                2.3.1
spglib                   2.0.0
stack-data               0.5.0
sympy                    1.12
terminado                0.13.0
testpath                 0.6.0
tinycss2                 1.1.1
torch                    2.1.2
torchvision              0.16.2
tornado                  6.2
traitlets                5.2.1.post0
triton                   2.1.0
websocket-client         1.4.2
Werkzeug                 2.2.2
widgetsnbextension       3.6.3
```

```bash

(env_pyt_ase_jpy) [pfstcb@login01(avon) ~]$ pip install cutensor-cu12
Collecting cutensor-cu12
  Downloading cutensor_cu12-2.0.2-py3-none-manylinux2014_x86_64.whl (156.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.9/156.9 MB 9.2 MB/s eta 0:00:00
Installing collected packages: cutensor-cu12
Successfully installed cutensor-cu12-2.0.2
WARNING: You are using pip version 22.0.4; however, version 24.1.2 is available.
You should consider upgrading via the '/gpfs/home/csc/pfstcb/env_pyt_ase_jpy/bin/python -m pip install --upgrade pip' command.
(env_pyt_ase_jpy) [pfstcb@login01(avon) ~]$ pip list
Package                  Version
------------------------ -----------
anyio                    3.6.1
argon2-cffi              20.1.0
ase                      3.22.1
ase-ext                  20.9.0
asgiref                  3.5.2
asttokens                2.0.8
async-generator          1.10
backcall                 0.2.0
beautifulsoup4           4.10.0
beniget                  0.4.1
bleach                   5.0.1
Bottleneck               1.3.4
cachelib                 0.9.0
cutensor-cu12            2.0.2
cycler                   0.11.0
deap                     1.3.1
debugpy                  1.4.1
defusedxml               0.7.1
deprecation              2.1.0
entrypoints              0.4
executing                1.0.0
fastjsonschema           2.16.1
Flask                    2.2.2
Flask-Cors               3.0.10
Flask-Session            0.4.0
fonttools                4.34.0
gast                     0.5.3
ipykernel                6.13.0
ipython                  8.5.0
ipython-genutils         0.2.0
ipywidgets               7.7.4
itsdangerous             2.1.2
jedi                     0.18.1
json5                    0.9.10
jupyter-client           7.3.1
jupyter-core             4.10.0
jupyter-packaging        0.12.0
jupyter-server           1.21.0
jupyterlab               3.5.0
jupyterlab-pygments      0.1.2
jupyterlab-server        2.13.0
jupyterlab-widgets       1.1.3
kiwisolver               1.4.3
lxml                     4.9.1
matplotlib               3.5.2
matplotlib-inline        0.1.2
mistune                  0.8.1
mpi4py                   3.1.3
mpmath                   1.2.1
nbclassic                0.4.8
nbclient                 0.6.3
nbconvert                6.5.3
nbformat                 5.4.0
nest-asyncio             1.5.5
networkx                 3.2.1
notebook                 6.4.0
notebook-shim            0.1.0
numexpr                  2.8.1
numpy                    1.22.3
nvidia-cublas-cu12       12.1.3.1
nvidia-cuda-cupti-cu12   12.1.105
nvidia-cuda-nvrtc-cu12   12.1.105
nvidia-cuda-runtime-cu12 12.1.105
nvidia-cudnn-cu12        8.9.2.26
nvidia-cufft-cu12        11.0.2.54
nvidia-curand-cu12       10.3.2.106
nvidia-cusolver-cu12     11.4.5.107
nvidia-cusparse-cu12     12.1.0.106
nvidia-nccl-cu12         2.18.1
nvidia-nvjitlink-cu12    12.3.101
nvidia-nvtx-cu12         12.1.105
pandas                   1.4.2
pandocfilters            1.5.0
parso                    0.8.3
pickleshare              0.7.5
Pillow                   9.1.1
pip                      22.0.4
ply                      3.11
prometheus-client        0.11.0
prompt-toolkit           3.0.31
pure-eval                0.2.2
pybind11                 2.9.2
pytest-mock              3.8.2
pythran                  0.11.0
pyzmq                    23.2.1
SciPy                    1.8.1
Send2Trash               1.8.0
setuptools               58.1.0
sniffio                  1.3.0
soupsieve                2.3.1
spglib                   2.0.0
stack-data               0.5.0
sympy                    1.12
terminado                0.13.0
testpath                 0.6.0
tinycss2                 1.1.1
torch                    2.1.2
torchvision              0.16.2
tornado                  6.2
traitlets                5.2.1.post0
triton                   2.1.0
websocket-client         1.4.2
Werkzeug                 2.2.2
widgetsnbextension       3.6.3
```


To test the script you will need to run the following:

test_script.py
```python
try:
    from ase import Atoms
    print("ase available")
except ImportError:
    print("ase not available")
    raise
try:
    import torch
    print("torch reports {torch.cuda.is_available()}")
except ImportError:
    print("torch not available")
    raise
try:
    from cutensor.torch import EinsumGeneral
    print("cutensor.torch.EinsumGeneral available")
except ImportError:
    print("cutensor.torch.EinsumGeneral not available")
    raise
```

Either using vim to write it directly or scp to make it locally and copy it over make sure that file is in your home on avon.

Then create this file in your home as well.

The rest of this information is about submitting jobs more on this later for now copy and paste verbatim.

test_env.sbatch
```bash
#!/bin/bash
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=3700
#SBATCH --time=00:10:00
#SBATCH --partition=gpu
#SBATCH --gres=gpu:quadro_rtx_6000:1

module purge && module restore mod_pyt_ase_jpy

source env_pyt_ase_jpy/bin/activate

python test_script.py

jupyter-lab --no-browser
```

Submit this file using:

```bash
sbatch test_env.sbatch
```

You will need to keep an eye on your job like this

```bash
squeue --user=<scrtpUserName> --start
```

Then when the job starts make note of the node and run:

```bash
ssh -J <scrtpUserName>@avon.scrtp.warwick.ac.uk -N -L 8888:127.0.0.1:8888 scrtpUserName@<nodeXXX>.avon.hpc
```

Then check the output file which will be of the form slurm-xxxxxx.out (where xxxxxx is the SLURM job ID returned when submitting the batch script).
In this file you will find your token. Paste that into a browser and we can use jupyter.
Launch a notebook, and paste and run the test python code as we used above.

```python
try:
    from ase import Atoms
    print("ase available")
except ImportError:
    print("ase not available")
    raise
try:
    import torch
    print("torch reports {torch.cuda.is_available()}")
except ImportError:
    print("torch not available")
    raise
try:
    from cutensor.torch import EinsumGeneral
    print("cutensor.torch.EinsumGeneral available")
except ImportError:
    print("cutensor.torch.EinsumGeneral not available")
    raise
```

If that goes well use ctrl-c to end the notebook and quit to end the HPC job.